In [1]:
import itertools
import numpy as np
import math
#import matplotlib
import matplotlib.pyplot as plt
#import numba as nb
import threading as thd
#import logging as lg
from util_tools.operators import *
from util_tools.update_funcs import *
#problem constants
var_dict = {}
nu_c=var_dict['nu_c']=1e-6
mu_c=var_dict['mu_c']=1e-3
rho_c=var_dict['rho_c']=1e+3
var_dict['st_coef']=0.06
#real timestep
dt=var_dict['dt']=0.000001
gradP=var_dict['gradP']=-2.4
from tqdm import tqdm
n_iter=0
global epstot
'''
node generation section
'''
#domain length

Lx1=var_dict['Lx1']=0.02
Lx2=var_dict['Lx2']=0.01

r_dpl=var_dict['r_dpl']=var_dict['Lx2']/4
#number of cells on each direction
Nx1=var_dict['Nx1']=120
Nx2=var_dict['Nx2']=60

cell_vol=var_dict['cell_vol']=(Lx1/Nx1)*(Lx2/Nx2)

#mesh spacing
h=var_dict['h']=Lx1/Nx1

#redistancing pseudo-time count
tau=0.0
#redistancing pseudo-time step
dtau= var_dict['dtau']= 0.5*h

#smoothing range
M=var_dict['M']=3.0
#uave
u1_ave=var_dict['u1_ave']=0.02


#Initialization
pn,pnn,un,us,vn,vs,unn,vnn,rho,mu,phin,phis,phinn,u_ref=[np.zeros([Nx1+2,Nx2+2])]*14


#half_index_grid
hig_x, hig_y = np.meshgrid((Lx1/Nx1)*np.linspace(-1, Nx1+1, Nx1+3), (Lx2/Nx2)*np.linspace(-1, Nx2+1, Nx2+3),indexing='ij')
#full_index_grid
xi=np.zeros([Nx1+2,Nx2+2])
xj=np.zeros([Nx1+2,Nx2+2])
xi[0:Nx1+2,0:Nx2+2]=0.25*(hig_x[0:Nx1+2,0:Nx2+2]+hig_x[0+1:Nx1+2+1,0:Nx2+2]+hig_x[0:Nx1+2,0+1:Nx2+2+1]+hig_x[0+1:Nx1+2+1,0+1:Nx2+2+1])
xj[0:Nx1+2,0:Nx2+2]=0.25*(hig_y[0:Nx1+2,0:Nx2+2] + hig_y[0+1:Nx1+2+1,0:Nx2+2] + hig_y[0:Nx1+2,0+1:Nx2+2+1]+hig_y[0+1:Nx1+2+1,0+1:Nx2+2+1])


In [2]:
#lvlset init
phin[0:Nx1+2,0:Nx2+2]=lvlset_init(xi[0:Nx1+2,0:Nx2+2], xj[0:Nx1+2,0:Nx2+2],var_dict)




rho[0:Nx1+2,0:Nx2+2]=rho_distr(phin[0:Nx1+2,0:Nx2+2],var_dict)
mu[0:Nx1+2,0:Nx2+2]=mu_distr(phin[0:Nx1+2,0:Nx2+2],var_dict)

un[0:Nx1+2,0:Nx2+2]=ref_vel_prof(cell_cent_y[0:Nx1+2,0:Nx2+2])




In [5]:
phinn.shape

(62, 32)

In [ ]:
for yt in tqdm(range(8)):
    
    rho[0:Nx1+2,0:Nx2+2]=rho_distr(phin[0:Nx1+2,0:Nx2+2],var_dict)
    mu[0:Nx1+2,0:Nx2+2]=mu_distr(phin[0:Nx1+2,0:Nx2+2],var_dict)
    us=us_looper(un,vn, mu, rho, us,var_dict)
    us=BC_looper(us,var_dict)
    res=1000
    while res>1e-3:
        pnn=p_looper( us, vs, pn,rho, pnn,var_dict)
        res=LA.norm(pnn-pn)
        pn=pnn.copy()
    pnn=BC_looper(pnn,var_dict)
    unn=unn_looper(us, pnn, unn, rho,var_dict)
    unn=BC_looper(unn,var_dict)

    phis=phis_looper(phis,phin,un,vn,var_dict)
    phis=BC_looper(phis,var_dict)

    phinn=phinn_looper(phinn,phis,phin,un,us,vs,vn,var_dict)
    phinn=BC_looper(phinn,var_dict)

    phin=phinn.copy()

    un=unn.copy()
    var_dict['dt']=var_dict['dt']/5



In [ ]:
bub=plt.Circle((0.01, 0.005), r_dpl, color='grey', fill=False)
fig, ax=plt.subplots()
plt.contourf(cell_cent_x, cell_cent_y, phin, 20, cmap='inferno')
plt.colorbar()
ax.add_artist(bub)
plt.xlabel('$x_1$ (m)')
plt.ylabel('$x_2$ (m)')
plt.title('domain initial level-set, '+str(Nx2), fontsize=9)
plt.gca().set_aspect('equal')
plt.show() 



In [ ]:
plt.contourf(cell_cent_x, cell_cent_y, pnn, 20, cmap='inferno')
plt.colorbar()
plt.xlabel('$x_1$ (m)')
plt.ylabel('$x_2$ (m)')
plt.title('domain $p$ contour ($m/s$)')
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
plt.contourf(cell_cent_x, cell_cent_y, unn, 20, cmap='inferno')
plt.colorbar()
plt.xlabel('$x_1$ (m)')
plt.ylabel('$x_2$ (m)')
plt.title('domain $u_1$ contour ($m/s$)')
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
rho[0:Nx1+2,0:Nx2+2]=rho_distr(phin[0:Nx1+2,0:Nx2+2],var_dict)
mu[0:Nx1+2,0:Nx2+2]=mu_distr(phin[0:Nx1+2,0:Nx2+2],var_dict)
us=us_looper(un,vn, mu, rho, us,var_dict)
us=us_BC_looper(us,var_dict)
res=1000
while res>1e-3:
    pnn=p_looper( us, vs, pn,rho, pnn,var_dict)
    res=LA.norm(pnn-pn)
    pn=pnn.copy()
pnn=p_BC_looper(pnn,var_dict)
unn=unn_looper(us, pnn, unn, rho,var_dict)
unn=unn_BC_looper(unn,var_dict)

phis=phis_looper(phis,phin,un,vn,var_dict)
phis=phis_BC_looper(phis,var_dict)

phinn=phinn_looper(phinn,phis,phin,un,us,vs,vn,var_dict)
phinn=phinn_BC_looper(phinn,var_dict)

phin=phinn.copy()

un=unn.copy()
var_dict['dt']=var_dict['dt']/5